In [ ]:
!pip install -q starrocks "sqlalchemy<3.0"

In [ ]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://server:8181/catalog"
STARROCKS_URI = "starrocks://root@starrocks:9030"
KEYCLOAK_TOKEN_ENDPOINT = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"
WAREHOUSE = "demo"

CLIENT_ID = "starrocks"
CLIENT_SECRET = "X5IWbfDJBTcU1F3PGZWgxDJwLyuFQmSf"

# Create Starrocks Catalog

In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine(STARROCKS_URI)

In [ ]:
# Until https://github.com/StarRocks/starrocks/issues/50585 is fixed, we have to configure
# the credentials locally if we need a custom s3 endpoint.
with engine.connect() as connection:
    connection.execute(text("DROP CATALOG IF EXISTS lakekeeper"))
    connection.execute(
        text(f"""
        CREATE EXTERNAL CATALOG lakekeeper
        PROPERTIES
        (
            "type" = "iceberg",
            "iceberg.catalog.type" = "rest",
            "iceberg.catalog.uri" = "{CATALOG_URL}",
            "iceberg.catalog.warehouse" = "{WAREHOUSE}",
            "iceberg.catalog.oauth2-server-uri" = "{KEYCLOAK_TOKEN_ENDPOINT}",
            "iceberg.catalog.credential" = "{CLIENT_ID}:{CLIENT_SECRET}",
            "iceberg.catalog.scope" = "lakekeeper offline_access",
            "aws.s3.region" = "local",
            "aws.s3.enable_path_style_access" = "true",
            "aws.s3.endpoint" = "http://minio:9000",
            "aws.s3.access_key" = "minio-root-user",
            "aws.s3.secret_key" = "minio-root-password"
        )
        """)
    )
    connection.execute(text("SET CATALOG lakekeeper"))

## Read and Write Tables

In [ ]:
with engine.connect() as connection:
    databases = connection.execute(text("SHOW DATABASES")).fetchall()
    if ("starrocks_namespace",) not in databases:
        connection.execute(text("CREATE DATABASE starrocks_namespace"))
    
    connection.execute(
        text("CREATE TABLE IF NOT EXISTS starrocks_namespace.my_table (my_ints INT, my_floats DOUBLE, strings VARCHAR)")
    )
    connection.execute(
        text("INSERT INTO starrocks_namespace.my_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')")
    )
    result = connection.execute(
        text("SELECT * FROM starrocks_namespace.my_table"),
    ).fetchall()

print(result)